In [1]:
!pip install -q langchain-google-genai python-dotenv langchain-core

In [2]:
"""
Prompt Templates with LangChain and Google Gemini
================================================

In this notebook, we learn:
- Creating reusable prompt templates
- Using variables in prompts
- Difference between PromptTemplate and ChatPromptTemplate
- Formatting and using templates effectively

Official documentation:
- LangChain Prompt Templates: https://python.langchain.com/docs/concepts/prompt_templates/
- LangChain Templates Guide: https://python.langchain.com/docs/how_to/prompts_composition/
"""

from dotenv import load_dotenv
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    PromptTemplate,
)
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables
load_dotenv()

# Initialize the model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    max_tokens=10000,
)

✅ Gemini 2.5 Flash model initialized

## 1. BASIC PROMPT TEMPLATE

\n📝 1. Basic Prompt Template

In [4]:

# Create a simple template with one variable
basic_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple terms for a beginner."
)

# Format the template
formatted_prompt = basic_template.format(topic="machine learning")
print(f"Formatted prompt: {formatted_prompt}")

# Use with LLM
response = llm.invoke(formatted_prompt)
print(f"Response: {response.content}")

# =============================================================================

Formatted prompt: Explain machine learning in simple terms for a beginner.
Response: Imagine you want to teach a computer something, but instead of giving it a super detailed, step-by-step instruction manual for *every single possibility*, you let it figure things out for itself by showing it examples. That's essentially what Machine Learning (ML) is!

Let's break it down with a simple analogy:

**Think about how a child learns what a "dog" is:**

1.  **You show the child many pictures of dogs.** Some are big, some are small, different breeds, different colors. You say, "This is a dog." (This is your **Data**).
2.  **You also show them pictures of other animals** – cats, birds, cows – and say, "This is *not* a dog." (More **Data**, helping them understand what *isn't* a dog).
3.  **The child's brain starts to find patterns.** It learns that dogs usually have fur, four legs, a tail, certain types of faces, and they bark. (This is the **Learning Algorithm** at work).
4.  **After seeing m

## 2. MULTI-VARIABLE PROMPT TEMPLATE

\n📝 2. Multi-variable Prompt Template

In [5]:

# Create template with multiple variables
multi_template = PromptTemplate(
    input_variables=["subject", "audience", "length"],
    template="""Write a {length} explanation of {subject} for {audience}.
Make sure to use appropriate language and examples."""
)

# Format with multiple variables
formatted_multi = multi_template.format(
    subject="artificial intelligence",
    audience="high school students",
    length="short"
)

print(f"Formatted prompt: {formatted_multi}")

response = llm.invoke(formatted_multi)
print(f"Response: {response.content}")

# =============================================================================

Formatted prompt: Write a short explanation of artificial intelligence for high school students.
Make sure to use appropriate language and examples.
Response: ## What is Artificial Intelligence (AI)?

Imagine you're trying to teach a really smart robot how to play a game, like chess or even just recognize a cat in a picture. Instead of telling it every single move or every single detail of what makes a cat a cat, you show it lots and lots of examples. Over time, the robot starts to figure out the patterns and rules on its own.

That, in a nutshell, is Artificial Intelligence (AI)!

**AI is when computers are programmed to "think" and "learn" in ways that mimic human intelligence.** It's about creating machines that can:

1.  **Understand and process information:** Like understanding your voice commands or recognizing objects in a photo.
2.  **Learn from experience:** Just like you get better at a game the more you play, AI systems improve by analyzing tons of data.
3.  **Solve problems

## 3. CHAT PROMPT TEMPLATE

\n💬 3. Chat Prompt Template

In [6]:

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. Always provide clear, well-commented code examples."),
    ("human", "Show me how to {task} in Python.")
])

# Format the chat template
formatted_chat = chat_template.format_messages(task="read a CSV file")
print(f"Formatted messages: {formatted_chat}")

# Use with LLM
response = llm.invoke(formatted_chat)
print(f"Response: {response.content}")

# =============================================================================

Formatted messages: [SystemMessage(content='You are a helpful coding assistant. Always provide clear, well-commented code examples.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Show me how to read a CSV file in Python.', additional_kwargs={}, response_metadata={})]
Response: Reading CSV (Comma Separated Values) files in Python is a very common task. Python provides excellent built-in capabilities for this, and there's also a powerful external library, `pandas`, which is often preferred for data analysis.

Here, I'll show you how to read CSV files using both approaches:

1.  **Using Python's built-in `csv` module:** This is suitable for basic reading and writing, especially when you don't need extensive data manipulation.
2.  **Using the `pandas` library:** This is the industry standard for data analysis and manipulation in Python. It's highly optimized and provides powerful tools for working with tabular data.

---

### First, let's create a sample CSV file

Bef

## 4. FEW-SHOT PROMPT TEMPLATE

\n🎯 4. Few-shot Prompt Template

In [7]:

# Define examples for few-shot learning
examples = [
    {
        "input": "What is 2+2?",
        "output": "2+2 equals 4"
    },
    {
        "input": "What is the capital of France?",
        "output": "The capital of France is Paris"
    },
    {
        "input": "What color is the sky?",
        "output": "The sky is typically blue during clear weather"
    },
]

# Create example template
example_template = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

# Create few-shot prompt template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="You are a helpful assistant. Answer questions clearly and concisely.",
    suffix="Input: {input}\nOutput:",
    input_variables=["input"]
)

# Test the few-shot template
test_input = "What is the capital of Japan?"
formatted_prompt = few_shot_prompt.format(input=test_input)
print(f"Formatted few-shot prompt:\n{formatted_prompt}")
print("\n" + "-" * 50)

response = llm.invoke(formatted_prompt)
print(f"Response: {response.content}")

# =============================================================================

Formatted few-shot prompt:
You are a helpful assistant. Answer questions clearly and concisely.

Input: What is 2+2?
Output: 2+2 equals 4

Input: What is the capital of France?
Output: The capital of France is Paris

Input: What color is the sky?
Output: The sky is typically blue during clear weather

Input: What is the capital of Japan?
Output:

--------------------------------------------------
Response: The capital of Japan is Tokyo


## 5. CONDITIONAL TEMPLATE

\n🔄 5. Conditional Template Usage

In [8]:

# Create templates for different use cases
templates = {
    "explain": PromptTemplate(
        input_variables=["concept"],
        template="Provide a clear explanation of {concept} with real-world examples."
    ),
    "summarize": PromptTemplate(
        input_variables=["text"],
        template="Summarize the following text in 2-3 sentences: {text}"
    ),
    "translate": PromptTemplate(
        input_variables=["text", "language"],
        template="Translate the following text to {language}: {text}"
    )
}

# Function to use appropriate template


def process_request(request_type, **kwargs):
    if request_type in templates:
        template = templates[request_type]
        formatted_prompt = template.format(**kwargs)
        return llm.invoke(formatted_prompt)
    else:
        return "Unknown request type"


# Test different templates
explain_response = process_request("explain", concept="neural networks")
print(f"Explanation: {explain_response.content}")

summarize_response = process_request(
    "summarize",
    text="Artificial intelligence is a rapidly growing field that focuses on creating machines capable of performing tasks that typically require human intelligence. This includes learning, reasoning, problem-solving, and understanding natural language."
)
print(f"Summary: {summarize_response.content}")

# =============================================================================

Explanation: Imagine you want to teach a computer to recognize a cat. How would you do it? You could list rules: "It has pointy ears, whiskers, fur, a tail." But what about a cat curled up? Or seen from the side? Or a hairless cat? This rule-based approach quickly becomes complex and brittle.

This is where **Neural Networks** come in. They are a powerful type of machine learning algorithm inspired by the way the human brain works. Instead of being explicitly programmed with rules, they *learn* from examples.

---

## What is a Neural Network?

At its core, a neural network is a system of interconnected "neurons" (or "nodes") organized in layers. It's designed to recognize patterns and make predictions by learning from data.

Let's break down the key components:

1.  **The Neuron (Node): The Basic Building Block**
    *   Imagine a tiny decision-making unit.
    *   It receives several **inputs** (numbers).
    *   Each input is multiplied by a **weight** (another number). Weights dete

## 6. TEMPLATE COMPOSITION

\n🔧 6. Template Composition

In [9]:

# Create modular templates
context_template = PromptTemplate(
    input_variables=["domain"],
    template="You are an expert in {domain}."
)

task_template = PromptTemplate(
    input_variables=["task", "details"],
    template="Your task is to {task}. Here are the details: {details}"
)

# Combine templates


def create_composed_prompt(domain, task, details):
    context = context_template.format(domain=domain)
    task_prompt = task_template.format(task=task, details=details)
    return f"{context}\n\n{task_prompt}"


# Use composed template
composed_prompt = create_composed_prompt(
    domain="data science",
    task="analyze a dataset",
    details="The dataset contains customer purchase data with columns: date, product, price, customer_id"
)

print(f"Composed prompt: {composed_prompt}")

response = llm.invoke(composed_prompt)
print(f"Response: {response.content}")

# =============================================================================

Composed prompt: You are an expert in data science.

Your task is to analyze a dataset. Here are the details: The dataset contains customer purchase data with columns: date, product, price, customer_id
Response: Excellent! As an expert in data science, I'd approach this customer purchase dataset with a structured methodology, moving from data understanding to deep insights and actionable recommendations.

Here's my comprehensive analysis plan for a dataset with `date`, `product`, `price`, and `customer_id` columns:

---

## Data Analysis Plan: Customer Purchase Data

**Dataset:** Customer purchase records
**Columns:** `date`, `product`, `price`, `customer_id`

### Phase 1: Data Understanding & Preprocessing (EDA Foundation)

The first step is always to get a thorough understanding of the data's quality, structure, and basic characteristics.

1.  **Initial Data Inspection:**
    *   **Load Data:** Use Pandas (`pd.read_csv` or similar).
    *   **`df.head()` / `df.sample()`:** Glimpse th

## 7. KEY INFORMATION

📋 Key Information:

✅ PromptTemplate: For simple string templates

✅ ChatPromptTemplate: For structured conversations

✅ input_variables: Define template variables

✅ .format(): Fill in template variables

✅ .format_messages(): For chat templates

✅ Templates enable reusability and consistency

✅ Few-shot prompting improves response quality

## STUDENT TODO EXERCISE

In [10]:
print("\n" + "=" * 60)

📝 STUDENT TODO EXERCISE

In [11]:
print("=" * 60)

"""
TODO: Create a FewShotPromptTemplate for a sentiment analysis task

Requirements:
1. Create at least 4 examples showing different sentiments (positive, negative, neutral, mixed)
2. Each example should have:
   - "text": The input text to analyze
   - "sentiment": The sentiment classification
   - "confidence": A confidence score (high/medium/low)

3. Use an appropriate prefix that explains the task
4. Test your template with a new sentence

Example structure to get you started:
sentiment_examples = [
    {
        "text": "I love this product! It's amazing!",
        "sentiment": "positive",
        "confidence": "high"
    },
    # Add more examples here...
]

Hint: Think about edge cases like sarcasm or mixed feelings!
"""

# Your code here:
# sentiment_examples = [
#     # TODO: Add your examples
# ]

# sentiment_example_template = PromptTemplate(
#     # TODO: Define your template
# )

# sentiment_few_shot_prompt = FewShotPromptTemplate(
#     # TODO: Configure your few-shot template
# )

# Test your template:
# test_sentence = "The movie was okay, not great but watchable"
# response = llm.invoke(sentiment_few_shot_prompt.format(text=test_sentence))
# print(f"Sentiment analysis: {response.content}")

'\nTODO: Create a FewShotPromptTemplate for a sentiment analysis task\n\nRequirements:\n1. Create at least 4 examples showing different sentiments (positive, negative, neutral, mixed)\n2. Each example should have:\n   - "text": The input text to analyze\n   - "sentiment": The sentiment classification\n   - "confidence": A confidence score (high/medium/low)\n\n3. Use an appropriate prefix that explains the task\n4. Test your template with a new sentence\n\nExample structure to get you started:\nsentiment_examples = [\n    {\n        "text": "I love this product! It\'s amazing!",\n        "sentiment": "positive",\n        "confidence": "high"\n    },\n    # Add more examples here...\n]\n\nHint: Think about edge cases like sarcasm or mixed feelings!\n'